In [ ]:
import pandas as pd
import numpy as np
import re
import pickle
from sklearn import preprocessing 
from datetime import datetime

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.io as pio
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *

In [ ]:
# read clean datafile
df = pd.read_csv('data/dataset4.gz', compression='gzip', header=0, sep=',', quotechar='"')
print(df.shape)

## Create the rolling lag variable for time-series

In [ ]:
test=df.sort_values(by=['device','date'], ascending=True)
test.head()

In [ ]:
cols_2_lag=df.columns.drop(['device', 'date', 'failure', 'prefix','obs_counter', 'ndays', 'date_gap',
       'any_gap'])

In [ ]:
cols_2_lag

In [ ]:
for col in cols_2_lag:
    for i in range(1,5):
        df[f'{col}_lag0{i}'] = df.groupby('device')[col].shift(i)
        df.loc[df[f'{col}_lag0{i}'].isnull(), f'{col}_lag0{i}']=0

## Zip the data file

In [ ]:
df.to_csv('data/cleaned_eda.gz', compression='gzip', index=False)
print(df.shape)

In [ ]:
df = pd.read_csv('data/cleaned_eda.gz', compression='gzip', header=0, sep=',', quotechar='"')

In [ ]:
df.head()

In [ ]:
pd.options.display.max_seq_items = 2000
df.columns